In [11]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])
# without tanh is worse-off

In [30]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [4]:
train_gen = SequenceGen(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGen(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [61]:
convNets = Sequential(ConvBatchLeaky1D(1,2,5,stride=2),
                      ConvBatchLeaky1D(2,4,5,stride=2),
                      ConvBatchLeaky1D(4,8,5,stride=2),
                      ConvBatchLeaky1D(8,16,5,stride=2),
                      ConvBatchLeaky1D(16,32,5,stride=2),
                      Dropout(0.5),                      
                      ResidualBlockGLU(32),
                      Dropout(0.5),                      
                      ResidualBlockGLU(64),
                      Dropout(0.5),
                      ResidualBlockGLU(128),
                      Dropout(0.5),
                      ResidualBlockGLU(256),
                      Dropout(0.5),
                      GRU_NCL(512,512,batch_first=True,bidirectional=True,returnH=True))

In [62]:
linear = Sequential(Dropout(0.5),Linear(512*2,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [63]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [64]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [65]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [66]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.8848903826796093, val_loss:2.774271249771118
epoch:1, train_loss:2.446963068384391, val_loss:2.6067757606506348
epoch:2, train_loss:2.3830984389552703, val_loss:2.4521567821502686
epoch:3, train_loss:2.3700233674966373, val_loss:2.658595323562622
epoch:4, train_loss:2.3501817286014557, val_loss:2.7319726943969727
Training completed in 155.9416801929474s
